In this tutorial, we study the family of SPDEs :

$$\left(\frac{\partial}{\partial t} + c\left((1-\nabla H. \nabla)^{\alpha} + v.\nabla\right)\right)Z(s,t)=\sqrt{c}W_T(t)\otimes X_S(s)$$

where 

- $\nu>0$ is a time scale parameter
- $H$ is an anisotropic matrix 
- $v$ is a velocity vector
- $W_T$ is a temporal white-noise
- $X_S$ is a spatially structured  noise

In [ ]:
import numpy as np
import gstlearn as gl


In [ ]:
from sksparse.cholmod import cholesky
import scipy as sc
from scipy.sparse import diags

In [ ]:
scale = 4.
kappa2 = 1./scale**2
dt = .1
c = 10
sqc = np.sqrt(c)
sqdt = np.sqrt(dt)

In [ ]:
nx = [100,100]
m = gl.Model.createFromParam(gl.ECov.BESSEL_K,range=1,param=2,flagRange=False)
mesh = gl.MeshETurbo(nx ,[1,1])
S = gl.ShiftOpCs(mesh,m)
St = gl.csToTriplet(S.getS())
Smat=sc.sparse.csc_matrix((np.array(St.values), (np.array(St.rows), np.array(St.cols))),
                          shape=(St.nrows,St.ncols))
TildeC = diags(np.sqrt(S.getTildeC()))
G = TildeC @ Smat @ TildeC
M = TildeC @ TildeC
K = (kappa2 * M + G) @ (kappa2 * M + G) @ (kappa2 * M + G)
P = M +  c*dt * K
cholP = cholesky(P)

def evalInvA(x) :
    return  cholP.solve_A(M @ x)

def evalInvB(x) :
    return sqc * sqdt * cholP.solve_A(TildeC @ x)

In [ ]:
x = np.random.normal(size=Smat.shape[0])

for i in range(100):
    u = np.random.normal(size=Smat.shape[0])
    x = evalInvA(x) + evalInvB(u)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
#x = np.random.normal(size=Smat.shape[0])
xtot = np.zeros(shape = [Smat.shape[0],50])
xtot[:,0] = x
fig, ax = plt.subplots(figsize=(3,3))
ln = plt.imshow(x.reshape(nx), 'BrBG')

def init():
    ln.set_data(x.reshape(nx))
    return ln

def update(frame):
    u = np.random.normal(size=Smat.shape[0])
    xtot[:,frame] = evalInvA(xtot[:,frame-1]) + evalInvB(u)
    ln.set_data(xtot[:,frame].reshape(nx))
    return ln

ani = FuncAnimation(fig, update, frames = 50,
                    init_func=init, blit=False,interval=10)
plt.show()


In [ ]:
%matplotlib inline
plt.imshow(xtot[:,49].reshape(nx)[50:150,50:150])
plt.colorbar()
plt.show()
np.var(xtot[:,0])